In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models
from torchvision.models import *
from plotly import express as px
from collections import Counter
import numpy as np
import random
import multiprocessing

from modules.dataset import IntelImageClassificationDataset
from modules.utility import NotebookPlotter, InferenceSession, Evaluator, ISO_time, apply_pruning
from modules.trainer import Trainer
from modules.optuna_monashara import run_optuna
from modules.BufferDataset import ShuffleBufferDataset

torch.manual_seed(1)
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("Using Apple Metal (MPS) backend:", DEVICE)
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("Using CUDA GPU:", DEVICE)
else:
    DEVICE = torch.device("cpu")
    print("Using CPU:", DEVICE)

def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True  # for reproducibility
    torch.backends.cudnn.benchmark = False

set_seed(1)

Using Apple Metal (MPS) backend: mps


https://www.kaggle.com/datasets/puneet6060/intel-image-classification

In [49]:
# labels, values = zip(*Counter([item[1] for item in dataset.train_dataset]).items())
# fig = px.bar(x=labels, y=values, labels={'x': 'Categories', 'y': 'Counts'}, title='Distribution of Classes')
# fig.show()

| n | label |
| --- | --- |
| 0 | buildings |
| 1 | forest |
| 2 | glacier |
| 3 | mountain |
| 4 | sea |
| 5 | street |

NotebookPlotter.plot_dataset_item_interactive(dataset.train_dataset)

In [ ]:
choice = 1 # 1,2,3
freezeLayer = True
pretrained_Weights = True
prune_model = True
OPTUNA_MO = False
Multi_B = True

if choice != 5:
    dataset = IntelImageClassificationDataset(resize=(150,150))
else:
    dataset = IntelImageClassificationDataset(resize=(384,384))

# 80% train, 20% validation for training Optuna
train_size = int(0.8 * len(dataset.train_dataset))
val_size = len(dataset.train_dataset) - train_size
train_subset, val_subset = random_split(dataset.train_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(1))

def build_model():

  # SqueezeNet 1.1
  if choice == 1:
      if(pretrained_Weights):
        model = models.squeezenet1_1(weights=SqueezeNet1_1_Weights.DEFAULT)
      else:
        model = models.squeezenet1_1()

      num_features = model.classifier[1].in_channels
      kernel_size = model.classifier[1].kernel_size
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[1] = nn.Conv2d(num_features, 6, kernel_size)

  # MobileNetV3 Small
  elif choice == 2:
      if(pretrained_Weights):
        model = models.mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)
      else:
        model = models.mobilenet_v3_small()
      num_features = model.classifier[3].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[3] = nn.Linear(num_features, 6)

  if prune_model:
    model = apply_pruning(model, amount=0.3)

  return model

if OPTUNA_MO:

    model = build_model()

    best_params, best_model_state, study = run_optuna(
        model=model,
        train_subset=train_subset,
        val_subset=val_subset,
        TrainerClass=Trainer,
        n_trials=12,
        seed=1
    )

    print("▶ Per-epoch validation accuracy (best trial):")
    best_trial = study.best_trial
    for epoch, acc in sorted(best_trial.intermediate_values.items()):
        print(f"   Epoch {epoch:2d}: {acc * 100:.2f}%")

    print(f"\n▶ Best hyperparameters: {best_params}")
    print(f"▶ Best overall accuracy: {study.best_value * 100:.2f}%")

    model.load_state_dict(best_model_state)

    dataloader = DataLoader(batch_size=best_params["BS_SUGGEST"], shuffle=True)
    dataloader = DataLoader(dataset.train_dataset, batch_size=best_params["BS_SUGGEST"], shuffle=True)
    trainer = Trainer(model=model,lr=best_params["LR_SUGGEST"],device=DEVICE)
    epochs = best_params["EPOCHS"]

    ''' BS_SUGGEST': 32, 'LR_SUGGEST': 8.841926348917726e-05, 'EPOCHS': 25 suggested from the OPTUNA
        and achieve the accuracy of 86.7 % on Testdata.'''

else:
    model = build_model()
    dataloader = DataLoader(dataset.train_dataset, batch_size=32, shuffle=True)
    trainer = Trainer(model=model, lr=8.841926348917726e-05, device=DEVICE)
    epochs = 25

if Multi_B:
    workers = max(1, multiprocessing.cpu_count() // 2)

    dataloader = DataLoader(
        dataset=dataloader.dataset,
        batch_size=dataloader.batch_size,
        shuffle=True,
        num_workers=workers,
        pin_memory=True,
        persistent_workers=True,
    )

In [51]:
# model.load_state_dict(torch.load(f"checkpoints/.pt"))
trainer.train(dataloader, epochs=epochs, silent=False)

  0%|          | 0/25 [00:00<?, ?it/s]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0

In [52]:
session = InferenceSession(model)
output = session(torch.stack(tuple(item[0] for item in dataset.test_dataset)))
Evaluator.acc(output, torch.tensor(tuple(item[1] for item in dataset.test_dataset))).item()

0.8670400381088257